## GRID3 User Survey Raw Aggregate Responses

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Markdown,display

def printmd(string):
    display(Markdown(string))

In [2]:
raw_df = pd.read_excel("mel_survey_data_analysis.xlsx")

In [3]:
def clean_countries(df):
    import string
    for i in [0,15]:
        df.iloc[:,i] = df.iloc[:,i].apply(lambda x: string.capwords(x.lower()))
    #combining the same object encoded differently
    df.iloc[:,0] = df.iloc[:,0].replace("Guinée","Guinee Conakry")
    return df

In [4]:
main_col_names = [
    'In which country do you work?',
    'Do you use geospatial software and datasets (json, shp, kml, gpx, etc...) in your work?',
    'Which of the following best characterizes your relationship with GRID3?',
    'How did you become familiar with GRID3-supported products and work (geospatial data, analysis, maps, training)?',
    'Please specify "other"',
    'GRID3 enables stakeholders to access geospatial information that is of a higher quality (or otherwise improved), than what is currently available outside the program',
    'GRID3 fills gaps or complements existing mechanisms for geospatial data collection, creation, and production',
    'Have you used GRID3-supported products (geospatial data, maps, analyses) in your professional work?',
    'How likely are you to use GRID3-supported products (geospatial data, maps, analyses) in the future?',
    'Overall, what is your opinion of GRID3?',
    'Do you know anyone outside your organization who is using GRID3-supported products (geospatial data, maps, analyses) in their professional work?']

In [5]:
branch1_names = [
    'What do you think are the MOST useful areas of GRID3 support?',
    'What do you think are the LEAST useful areas of GRID3 support?',
]

branch2_names = [
    'Have you used GRID3-supported products (geospatial data, maps, analyses) in your professional work?'
    'When did you first start engaging with or using GRID3-supported products (geospatial data, analysis, maps)?'
    'Please briefly describe the project(s) for which you used GRID3-supported products',
    'Which GRID3-supported geospatial data were used?',
    '',
    '',
    'Have you used GRID3-supported maps or data hubs?',
    'Please select the products you or your organization have used',
    'Please rate the user-experience of GRID3-supported maps, relative to other maps or methods',
    'How would you characterize the user-experience (ease of use, design, relevance) of GRID3-supported hubs, relative to other data hubs?',
    'GRID3-supported products correctly identify the intended target LOCATIONS for my organization’s program or service',
    'GRID3-supported products correctly identify the intended target POPULATION for my organization’s program or service',
    'What, if any, barrier(s) prevent the use of GRID3-supported products (geospatial data, maps, analyses)?']

In [6]:
#cleaning countries
working_df = clean_countries(raw_df)

In [7]:
working_df.iloc[:,94] = working_df.iloc[:,94].replace('Not sure','Not Sure')

In [8]:
def search_col_idx(df,name):
    return df.columns.get_loc(name)

In [9]:
def get_name_idx_pair(df,lst):
    ret_lst = []
    for name in lst:
        idx = search_col_idx(df,name)
        ret_lst.append((name,idx))
    return ret_lst

In [10]:
main_cols = get_name_idx_pair(working_df,main_col_names)

In [11]:
def get_df_from_info(info):
    return info[2]

In [47]:
def join_multiple_res(lst_infos,start,stop,parse_name=True):
    selected = lst_infos[start:stop]
    #print(selected[2][2])
    df = pd.DataFrame()
    for item in selected:

        res_df = get_df_from_info(item)
        
        if parse_name:
        #parse the response name from the question
            name = ''.join(item[1][item[1].find('?/'):][2:])
            if not name:
                import re 
                pattern = re.compile(r'(are|is) (.*),')
                name = ' || '.join((pattern.search(item[1]).group(2).strip(),f'Total Count: {item[3]}'))
                #print(name)
                try:
                    res_df.columns = pd.MultiIndex.from_product([[name],res_df.columns])
                except:
                    pass

            name = ' || '.join((item[1][item[1].find('?/'):][2:],f'Total Count: {item[3]}'))
            #renaming the multilevel index
            try:
                res_df.columns = pd.MultiIndex.from_product([[name],res_df.columns])
            except:
                pass
        else:
            res_df.columns = pd.MultiIndex.from_product([[item[1]],res_df.columns])
        
        #print(name)
        #display(res_df)
        df = pd.concat([df,res_df],axis=1)
        #print(res_df)
    return df

In [13]:
def get_response_info_per_question(df,col_idx):
    # getting the proportion of responses 
    ser_name = df.columns[col_idx]
    pct_series = df.iloc[:,col_idx].value_counts(normalize=True).rename('% Responses',inplace=True)
    pct_series = pct_series.apply(lambda x: "{:.2%}".format(x))
    
    # getting the raw counts of responses
    cnt_series = df.iloc[:,col_idx].value_counts().rename('# Responses',inplace=True)
    
    # creating the df
    ret_df = pd.concat([pct_series,cnt_series],axis=1)
    count = df.iloc[:,col_idx].count()
    #ret_df.loc['Total Counts'] = [None,count]
    
    return (ser_name,ret_df,count)

In [14]:
def get_all_responses(df,cols):
    ret_lst = []
    try:
        for _,idx in cols:
            info = get_response_info_per_question(df,idx)
            ret_lst.append((idx,*info))
    except:
        for idx in cols:
            info = get_response_info_per_question(df,idx)
            ret_lst.append((idx,*info))
    return ret_lst

In [15]:
def clean_name(string):
    return string.replace('?','').replace('/','-').replace('*','')[:30]

In [16]:
main_col_info = get_all_responses(working_df,main_cols)

In [18]:
i = working_df.shape[1]
all_info = get_all_responses(working_df,zip([None]*i,list(range(i))))

In [20]:
pd.set_option('display.max_rows', None)

In [23]:
channel_range = list(range(4,13))
most_important_range = list(range(18,31))
least_important_range = list(range(33,46))
data_used_range = list(range(51,62))
barriers_range = list(range(103,115))

In [24]:
lst_clusters = [channel_range,most_important_range,least_important_range,data_used_range,barriers_range]

In [25]:
ignore_cols = [3,13,14,17,31,32,49,50,102,117,119,120,121,122].extend(lst_clusters)

In [50]:
most_important_df = working_df.loc[:,"What do you think are the **most** useful areas of GRID3 support?/Data collection":"What do you think are the **most** useful areas of GRID3 support?/Other"]
least_important_df = working_df.loc[:,"What do you think are the **least** useful areas of GRID3 support?/Data collection":"What do you think are the **least** useful areas of GRID3 support?/Other"]
channel_df = working_df.iloc[:,4:13]
data_used_df = working_df.iloc[:,51:62]
map_df = working_df.iloc[:,90:97]
barriers_df = working_df.iloc[:,103:115]

In [28]:
relative_rankings = working_df.loc[:,'How accurate are the dataset(s), relative to other similar geospatial data?':'How useful are GRID3-supported health catchment area datasets, relative to other geospatial health catchment area data?']

In [29]:
relative_accuracies = relative_rankings.iloc[:,::2]
relative_usefulness = relative_rankings.iloc[:,1::2]

In [45]:
def pprint_proportions_datagroup(df,parse_name=True):
    #takes a dataframe and returns the condensed summary of a dataframe
    size = df.shape[1]
    
    responses = get_all_responses(df,list(range(size)))
    joined_responses = join_multiple_res(responses,0,size,parse_name).fillna('---').T

    return joined_responses.reindex(sorted(joined_responses.columns,reverse=True), axis=1)

In [31]:
size_relative = relative_accuracies.shape[1]
size_usefulness = relative_usefulness.shape[1]

In [54]:
printed_channels = False
printed_important = False
printed_not_important = False
printed_4 = False
printed_5 = False
printed_6 = False
printed_7 = False

for col_idx, name,ser,count in all_info:
    if col_idx in [3,13,14,17,31,32,49,50,89,102,117,119,120,121,122]:
        continue
    if col_idx in range(4,13) and not printed_channels:
        name = name[:name.find('?/')]
        printmd(f"**{name}**")
        display(pprint_proportions_datagroup(channel_df))
        printed_channels = True
        print('\n')
    elif col_idx in range(4,13) and printed_channels:
        continue
    elif col_idx in range(18,31) and not printed_important:
        name = name[:name.find('?/')]
        printmd(f"**{name}** *Total Responses:{count}*")
        display(pprint_proportions_datagroup(most_important_df))
        printed_important = True
        print('\n')
    elif col_idx in range(18,31) and printed_important:
        continue
    elif col_idx in range(33,46) and not printed_not_important:
        name = name[:name.find('?/')]
        printmd(f"**{name}** *Total Responses:{count}*")
        display(pprint_proportions_datagroup(least_important_df))
        printed_not_important = True
        print('\n')
    elif col_idx in range(33,46) and printed_not_important:
        pass
    elif col_idx in range(51,62) and not printed_4:
        name = name[:name.find('?/')]
        printmd(f"**{name}** *Total Responses:{count}*")
        display(pprint_proportions_datagroup(data_used_df))
        printed_4 = True
        print('\n')
    elif col_idx in range(51,62) and printed_4:
        pass
    elif col_idx in range(63,84) and not printed_5:
        name = 'Usefulness and Accuracy of various GRID3 supported data'
        printmd(f"**{name}**")
        display(pprint_proportions_datagroup(relative_accuracies))
        display(pprint_proportions_datagroup(relative_usefulness))
        printed_5 = True
        print('\n')
    elif col_idx in range(63,84) and printed_5:
        pass
    elif col_idx in range(90,98) and not printed_6:
        name = 'Map Usefulness'
        printmd(f"**{name}**")
        display(pprint_proportions_datagroup(map_df,False))
        printed_6 = True
        print('\n')
    elif col_idx in range(90,97) and printed_6:
        pass
    elif col_idx in range(103,115) and not printed_7:
        name = name[:name.find('?/')]
        printmd(f"**{name}**")
        display(pprint_proportions_datagroup(barriers_df))
        printed_7 = True
        print('\n')
    elif col_idx in range(103,115) and printed_7:
        pass
    else:
        printmd(f"**{name}** *Total Responses:{count}*")
        display(ser)
        #print(col_idx)

**In which country do you work?** *Total Responses:137*

,% Responses,# Responses
Nigeria,27.74%,38
Zambia,20.44%,28
Drc,19.71%,27
Burkina Faso,4.38%,6
Sierra Leone,3.65%,5
Mozambique,3.65%,5
Ghana,2.19%,3
Kenya,2.19%,3
South Sudan,2.19%,3
Somalia,1.46%,2


**Do you use geospatial software and datasets (json, shp, kml, gpx, etc...) in your work?** *Total Responses:137*

,% Responses,# Responses
Yes,84.67%,116
No,15.33%,21


**Which of the following best characterizes your relationship with GRID3?** *Total Responses:137*

...

,% Responses,# Responses
The maps are not live. I.e. data does not change with time lapse (following year),14.29%,1
clear or detailed methodologies of products including metadata,14.29%,1
Limited computing power and slow download of products owing to poor internet access,14.29%,1
"Only a fraction of the DRC has been mapped, limiting the utility to only specific geographies",14.29%,1
Allow the application for production of population grid to work with land cover data and all its layers to achieve a more accurate results in DUG and TUC.,14.29%,1
I wish to continue the training (entire program),14.29%,1
\nN / A. I don't know how to rate this question!,14.29%,1


**How likely are you to use GRID3-supported products (geospatial data, maps, analyses) in the future?** *Total Responses:137*

,% Responses,# Responses
Very likely,58.39%,80
Likely,32.12%,44
Unsure,3.65%,5
Very unlikely,3.65%,5
Unlikely,2.19%,3


**Do you know anyone outside your organization who is using GRID3-supported products (geospatial data, maps, analyses) in their professional work?** *Total Responses:137*

,% Responses,# Responses
No,67.88%,93
Yes,32.12%,44
